### Datasets utilisés
- 1. Voitures particulières immatriculées par commune et par type de charge: https://www.data.gouv.fr/fr/datasets/voitures-particulieres-immatriculees-par-commune-et-par-type-de-recharge/
- 2. API découpage administratif des communes: https://geo.api.gouv.fr/decoupage-administratif/communes
- 3. Départements de France: https://www.data.gouv.fr/fr/datasets/departements-de-france/


## Objectif du notebook:
Récuprer les infos sur les communes: notamment département / région / population pour les merger avec les données des immatriculations par commune et par type de charge.

In [3]:
# Import des librairies
import requests
import json
import pandas as pd

In [4]:
# Importer les données des immatriculations
immatriculations = pd.read_csv('raw_data/voitures-rechargeables-par-commune.csv', sep=';', dtype={'codgeo': 'str'})
immatriculations

,codgeo,libgeo,epci,libepci,date_arrete,nb_vp_rechargeables_el,nb_vp_rechargeables_gaz,nb_vp
0,35081,CLAYES,243500139,Rennes Métropole,2022-06-30,9,0,821
1,35082,COËSMES,243500634,CC Roche aux Fées Communauté,2020-12-31,2,0,1533
2,35082,COËSMES,243500634,CC Roche aux Fées Communauté,2021-03-31,2,0,1534
3,35082,COËSMES,243500634,CC Roche aux Fées Communauté,2021-12-31,4,0,1545
4,35082,COËSMES,243500634,CC Roche aux Fées Communauté,2022-03-31,6,0,1543
...,...,...,...,...,...,...,...,...
316607,23100,JARNAGES,200067544,CC Creuse Confluence,2021-06-30,3,0,527
316608,23100,JARNAGES,200067544,CC Creuse Confluence,2022-03-31,3,0,534
316609,23100,JARNAGES,200067544,CC Creuse Confluence,2022-06-30,5,0,533
316610,23101,JOUILLAT,200034825,CA du Grand Guéret,2020-12-31,0,0,471


In [5]:
# Récupérer l'ensemble des communes de France

url = f"https://geo.api.gouv.fr/communes"

payload = {}
headers = {}

response = requests.request("GET", url, headers=headers, data=payload)

communes_list = json.loads(response.text)
communes_df = pd.DataFrame(communes_list)

In [6]:
communes_df

,nom,code,codeDepartement,siren,codeEpci,codeRegion,codesPostaux,population
0,L'Abergement-Clémenciat,01001,01,210100012,200069193,84,[01400],779.0
1,L'Abergement-de-Varey,01002,01,210100020,240100883,84,[01640],256.0
2,Ambérieu-en-Bugey,01004,01,210100046,240100883,84,[01500],14134.0
3,Ambérieux-en-Dombes,01005,01,210100053,200042497,84,[01330],1751.0
4,Ambléon,01006,01,210100061,200040350,84,[01300],112.0
...,...,...,...,...,...,...,...,...
35044,Touho,98830,988,NaN,NaN,988,[98831],2380.0
35045,Voh,98831,988,NaN,NaN,988,[98833],2856.0
35046,Yaté,98832,988,NaN,NaN,988,[98834],1667.0
35047,Kouaoua,98833,988,NaN,NaN,988,[98818],1304.0


In [7]:
print("nombre de codes de communes uniques dans le dataset des immatriculations: ", immatriculations['codgeo'].nunique())
print("nombre de codes de communes uniques dans le dataset des communes: ", communes_df['code'].nunique())

nombre de codes de communes uniques dans le dataset des immatriculations:  35189
nombre de codes de communes uniques dans le dataset des communes:  35049


In [12]:
# Commnes dans immatriculations mais pas dans communes_df
len(set(immatriculations['codgeo'].unique())-set(communes_df['code'].unique()))

235

In [14]:
# Commnes dans communes_df mais pas dans immatriculations
len(set(communes_df['code'].unique())-set(immatriculations['codgeo'].unique()))

95

In [15]:
df_code=pd.read_csv('raw_data/communes-departement-region.csv')
df_code.head()

,code_commune_INSEE,nom_commune_postal,code_postal,libelle_acheminement,ligne_5,latitude,longitude,code_commune,article,nom_commune,nom_commune_complet,code_departement,nom_departement,code_region,nom_region
0,1001,L ABERGEMENT CLEMENCIAT,1400,L ABERGEMENT CLEMENCIAT,NaN,46.153426,4.926114,1.0,L',Abergement-Clémenciat,L'Abergement-Clémenciat,1,Ain,84.0,Auvergne-Rhône-Alpes
1,1002,L ABERGEMENT DE VAREY,1640,L ABERGEMENT DE VAREY,NaN,46.009188,5.428017,2.0,L',Abergement-de-Varey,L'Abergement-de-Varey,1,Ain,84.0,Auvergne-Rhône-Alpes
2,1004,AMBERIEU EN BUGEY,1500,AMBERIEU EN BUGEY,NaN,45.960848,5.372926,4.0,NaN,Ambérieu-en-Bugey,Ambérieu-en-Bugey,1,Ain,84.0,Auvergne-Rhône-Alpes
3,1005,AMBERIEUX EN DOMBES,1330,AMBERIEUX EN DOMBES,NaN,45.996180,4.912273,5.0,NaN,Ambérieux-en-Dombes,Ambérieux-en-Dombes,1,Ain,84.0,Auvergne-Rhône-Alpes
4,1006,AMBLEON,1300,AMBLEON,NaN,45.749499,5.594320,6.0,NaN,Ambléon,Ambléon,1,Ain,84.0,Auvergne-Rhône-Alpes


**Comparaison à faire avec la table de dimension communes obtenue au-dessus.**

#### Observation
Il semble manquer environ 100 communes dans le dataset des communes...

In [6]:
# chercher les communes manquantes une par une
codes_immats = list(immatriculations['codgeo'].unique())
codes_communes = list(communes_df['code'].unique())
nb_manquantes = 0
communes_manquantes_trouvees = []
for code in codes_immats:
    if code not in codes_communes:
        nb_manquantes += 1
        url = f"https://geo.api.gouv.fr/communes/{code}"

        payload = {}
        headers = {}

        response = requests.request("GET", url, headers=headers, data=payload)
        
        if response.text != 'Not Found':
            communes_manquantes_trouvees.append(json.loads(response.text))
communes_manquantes_trouvees_df = pd.DataFrame(communes_manquantes_trouvees)
print(f"{len(communes_manquantes_trouvees)} communes ont été trouvées sur les {nb_manquantes} communes manquantes.")

45 communes ont été trouvées sur les 235 communes manquantes.


On peut ajouter ces communes trouvées au dataset.

In [7]:
# Ajouter les communes trouvées au dataset
communes_df = pd.concat([communes_df, communes_manquantes_trouvees_df])
communes_df

,nom,code,codeDepartement,siren,codeEpci,codeRegion,codesPostaux,population
0,L'Abergement-Clémenciat,01001,01,210100012,200069193,84,[01400],779.0
1,L'Abergement-de-Varey,01002,01,210100020,240100883,84,[01640],256.0
2,Ambérieu-en-Bugey,01004,01,210100046,240100883,84,[01500],14134.0
3,Ambérieux-en-Dombes,01005,01,210100053,200042497,84,[01330],1751.0
4,Ambléon,01006,01,210100061,200040350,84,[01300],112.0
...,...,...,...,...,...,...,...,...
40,Marseille 10e Arrondissement,13210,13,NaN,NaN,93,[13010],58491.0
41,Marseille 11e Arrondissement,13211,13,NaN,NaN,93,[13011],57278.0
42,Marseille 12e Arrondissement,13212,13,NaN,NaN,93,[13012],61406.0
43,Paris 10e Arrondissement,75110,75,NaN,NaN,11,[75010],86472.0


### Merger les 2 datasets: communes et immatriculations

In [8]:
# Merge les 2 datasets (immatriculations et communes) sur le code INSEE
immatriculations = immatriculations.merge(communes_df, left_on='codgeo', right_on='code', how='left')
immatriculations

,codgeo,libgeo,epci,libepci,date_arrete,nb_vp_rechargeables_el,nb_vp_rechargeables_gaz,nb_vp,nom,code,codeDepartement,siren,codeEpci,codeRegion,codesPostaux,population
0,35081,CLAYES,243500139,Rennes Métropole,2022-06-30,9,0,821,Clayes,35081,35,213500812,243500139,53,[35590],908.0
1,35082,COËSMES,243500634,CC Roche aux Fées Communauté,2020-12-31,2,0,1533,Coësmes,35082,35,213500820,243500634,53,[35134],1465.0
2,35082,COËSMES,243500634,CC Roche aux Fées Communauté,2021-03-31,2,0,1534,Coësmes,35082,35,213500820,243500634,53,[35134],1465.0
3,35082,COËSMES,243500634,CC Roche aux Fées Communauté,2021-12-31,4,0,1545,Coësmes,35082,35,213500820,243500634,53,[35134],1465.0
4,35082,COËSMES,243500634,CC Roche aux Fées Communauté,2022-03-31,6,0,1543,Coësmes,35082,35,213500820,243500634,53,[35134],1465.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316607,23100,JARNAGES,200067544,CC Creuse Confluence,2021-06-30,3,0,527,Jarnages,23100,23,212310007,200067544,75,[23140],439.0
316608,23100,JARNAGES,200067544,CC Creuse Confluence,2022-03-31,3,0,534,Jarnages,23100,23,212310007,200067544,75,[23140],439.0
316609,23100,JARNAGES,200067544,CC Creuse Confluence,2022-06-30,5,0,533,Jarnages,23100,23,212310007,200067544,75,[23140],439.0
316610,23101,JOUILLAT,200034825,CA du Grand Guéret,2020-12-31,0,0,471,Jouillat,23101,23,212310106,200034825,75,[23220],387.0


In [9]:
# Supprimer les colonnes redondantes
immatriculations = immatriculations.drop(columns=['nom', 'code', 'codeEpci'])
immatriculations

,codgeo,libgeo,epci,libepci,date_arrete,nb_vp_rechargeables_el,nb_vp_rechargeables_gaz,nb_vp,codeDepartement,siren,codeRegion,codesPostaux,population
0,35081,CLAYES,243500139,Rennes Métropole,2022-06-30,9,0,821,35,213500812,53,[35590],908.0
1,35082,COËSMES,243500634,CC Roche aux Fées Communauté,2020-12-31,2,0,1533,35,213500820,53,[35134],1465.0
2,35082,COËSMES,243500634,CC Roche aux Fées Communauté,2021-03-31,2,0,1534,35,213500820,53,[35134],1465.0
3,35082,COËSMES,243500634,CC Roche aux Fées Communauté,2021-12-31,4,0,1545,35,213500820,53,[35134],1465.0
4,35082,COËSMES,243500634,CC Roche aux Fées Communauté,2022-03-31,6,0,1543,35,213500820,53,[35134],1465.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
316607,23100,JARNAGES,200067544,CC Creuse Confluence,2021-06-30,3,0,527,23,212310007,75,[23140],439.0
316608,23100,JARNAGES,200067544,CC Creuse Confluence,2022-03-31,3,0,534,23,212310007,75,[23140],439.0
316609,23100,JARNAGES,200067544,CC Creuse Confluence,2022-06-30,5,0,533,23,212310007,75,[23140],439.0
316610,23101,JOUILLAT,200034825,CA du Grand Guéret,2020-12-31,0,0,471,23,212310106,75,[23220],387.0


In [10]:
# Merger avec les noms des départements et régions
departements = (pd.read_csv('raw_data/departements-france.csv')
    .rename(columns={'code_departement': 'codeDepartement'}).drop(columns=['code_region']))
immatriculations = immatriculations.merge(departements, on='codeDepartement', how='left')
immatriculations

,codgeo,libgeo,epci,libepci,date_arrete,nb_vp_rechargeables_el,nb_vp_rechargeables_gaz,nb_vp,codeDepartement,siren,codeRegion,codesPostaux,population,nom_departement,nom_region
0,35081,CLAYES,243500139,Rennes Métropole,2022-06-30,9,0,821,35,213500812,53,[35590],908.0,Ille-et-Vilaine,Bretagne
1,35082,COËSMES,243500634,CC Roche aux Fées Communauté,2020-12-31,2,0,1533,35,213500820,53,[35134],1465.0,Ille-et-Vilaine,Bretagne
2,35082,COËSMES,243500634,CC Roche aux Fées Communauté,2021-03-31,2,0,1534,35,213500820,53,[35134],1465.0,Ille-et-Vilaine,Bretagne
3,35082,COËSMES,243500634,CC Roche aux Fées Communauté,2021-12-31,4,0,1545,35,213500820,53,[35134],1465.0,Ille-et-Vilaine,Bretagne
4,35082,COËSMES,243500634,CC Roche aux Fées Communauté,2022-03-31,6,0,1543,35,213500820,53,[35134],1465.0,Ille-et-Vilaine,Bretagne
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316607,23100,JARNAGES,200067544,CC Creuse Confluence,2021-06-30,3,0,527,23,212310007,75,[23140],439.0,Creuse,Nouvelle-Aquitaine
316608,23100,JARNAGES,200067544,CC Creuse Confluence,2022-03-31,3,0,534,23,212310007,75,[23140],439.0,Creuse,Nouvelle-Aquitaine
316609,23100,JARNAGES,200067544,CC Creuse Confluence,2022-06-30,5,0,533,23,212310007,75,[23140],439.0,Creuse,Nouvelle-Aquitaine
316610,23101,JOUILLAT,200034825,CA du Grand Guéret,2020-12-31,0,0,471,23,212310106,75,[23220],387.0,Creuse,Nouvelle-Aquitaine


In [11]:
# Ajustements sur la structure du DataFrame
column_order = [
     'codgeo',
     'libgeo',
     'epci',
     'libepci',
     'date_arrete',
     'nb_vp_rechargeables_el',
     'nb_vp_rechargeables_gaz',
     'nb_vp',
     'code_departement',
     'nom_departement',
     'code_region',
     'nom_region',
     'codes_postaux',
     'population',
     'siren'
]
immatriculations = immatriculations.rename(columns={'codeDepartement': 'code_departement',
                                 'codeRegion': 'code_region',
                                 'codesPostaux': 'codes_postaux'})[column_order]
immatriculations

,codgeo,libgeo,epci,libepci,date_arrete,nb_vp_rechargeables_el,nb_vp_rechargeables_gaz,nb_vp,code_departement,nom_departement,code_region,nom_region,codes_postaux,population,siren
0,35081,CLAYES,243500139,Rennes Métropole,2022-06-30,9,0,821,35,Ille-et-Vilaine,53,Bretagne,[35590],908.0,213500812
1,35082,COËSMES,243500634,CC Roche aux Fées Communauté,2020-12-31,2,0,1533,35,Ille-et-Vilaine,53,Bretagne,[35134],1465.0,213500820
2,35082,COËSMES,243500634,CC Roche aux Fées Communauté,2021-03-31,2,0,1534,35,Ille-et-Vilaine,53,Bretagne,[35134],1465.0,213500820
3,35082,COËSMES,243500634,CC Roche aux Fées Communauté,2021-12-31,4,0,1545,35,Ille-et-Vilaine,53,Bretagne,[35134],1465.0,213500820
4,35082,COËSMES,243500634,CC Roche aux Fées Communauté,2022-03-31,6,0,1543,35,Ille-et-Vilaine,53,Bretagne,[35134],1465.0,213500820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316607,23100,JARNAGES,200067544,CC Creuse Confluence,2021-06-30,3,0,527,23,Creuse,75,Nouvelle-Aquitaine,[23140],439.0,212310007
316608,23100,JARNAGES,200067544,CC Creuse Confluence,2022-03-31,3,0,534,23,Creuse,75,Nouvelle-Aquitaine,[23140],439.0,212310007
316609,23100,JARNAGES,200067544,CC Creuse Confluence,2022-06-30,5,0,533,23,Creuse,75,Nouvelle-Aquitaine,[23140],439.0,212310007
316610,23101,JOUILLAT,200034825,CA du Grand Guéret,2020-12-31,0,0,471,23,Creuse,75,Nouvelle-Aquitaine,[23220],387.0,212310106


In [12]:
# Exporter le résultat
immatriculations.to_csv('enriched_data/voitures-rechargeables-par-commune-enrichies.csv', index=False)